In [1]:
!pip install google-generativeai
!pip install python-dotenv

In [32]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

In [33]:
load_dotenv()
api_key_1 = os.getenv('GOOGLE_API_KEY_1')
api_key_2 = os.getenv('GOOGLE_API_KEY_2')
api_key_3 = os.getenv('GOOGLE_API_KEY_3')
api_key_4 = os.getenv('GOOGLE_API_KEY_4')
api_key_5 = os.getenv('GOOGLE_API_KEY_5')
api_key_6 = os.getenv('GOOGLE_API_KEY_6')

api_keys = [
    api_key_1,
    api_key_2,
    api_key_3,
    api_key_4,
    api_key_5,
    api_key_6
]

In [35]:
filenames = [
    "group_1.txt",
    "group_2.txt",
    "group_3.txt",
    "group_4.txt",
    "group_5.txt",
    "group_6.txt",
    "group_7.txt",
    "group_8.txt",
    "group_9.txt",
    "group_10.txt",
    "group_11.txt",
    "group_12.txt",
]

In [36]:
file_contents = []

In [37]:
for filename in filenames:
    file = open(f"./data/{filename}", "r")
    file_content = file.read()
    file.close()
    file_contents.append(file_content)

In [38]:
question = "Kas ma võin autoga sõites alkoholi tarbida?"

In [39]:
system_prompt = "Sa oled seaduste abiline. Kui sulle antud seaduses on küsimusele vastust, siis vasta sellele. Kui vastus puudub, siis vasta '0'"

In [40]:
response_format = """kastuta vastamisel järgmist formaati:
<seaduse nimi>

<sinu vastus>"""

In [41]:
responses = []

In [42]:
group_index = 0
group_count = len(api_keys) * 2

for api_key in api_keys:
    genai.configure(api_key=api_key)

    print(f"progress: {group_index + 1}/{group_count}")
    file_content = file_contents[group_index]
    model = genai.GenerativeModel("gemini-1.5-flash")
    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text
    responses.append(response_text)

    group_index += 1

    print(f"progress: {group_index + 1}/{group_count}")
    file_content = file_contents[group_index]
    model = genai.GenerativeModel("gemini-1.5-flash-8b")
    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text
    responses.append(response_text)

    group_index += 1

progress: 1/12
progress: 2/12
progress: 3/12
progress: 4/12
progress: 5/12
progress: 6/12
progress: 7/12
progress: 8/12
progress: 9/12
progress: 10/12
progress: 11/12
progress: 12/12


In [43]:
for response in responses:
    print(response)

Võlaõigusseadus

0

<Liiklusseadus>

0

Karistusseadustik

Ei. Karistusseadustiku § 424 sätestab, et mootorsõiduki juhtimise eest joobeseisundis karistatakse rahalise karistuse või kuni kolmeaastase vangistusega.

<Meresõiduohutuse seadus>

0

Sotsiaalhoolekande seadus

0
<Liiklusseadus>

0

Autoveoseadus

Ei, see on keelatud. Autoveoseaduse § 97 lõige 1 sätestab, et autojuht ei tohi olla alkoholi tarvitanud.

<Liiklusseadus>

0

Tubakaseadus

0

0

Reklaamiseadus

0

Eesti Rahvusringhäälingu seadus

0

Patendivoliniku seadus

0

Tõestamisseadus

0

Põhiseaduslikkuse järelevalve kohtumenetluse seadus

0

Kinnisasja avalikes huvides omandamise seadus

0

Rahvusvahelise eraõiguse seadus

0

Lastekaitseseadus

0

Mõõteseadus

0

Vedelkütuse erimärgistamise seadus

0

Rahvusvahelise eraõiguse seadus

0

Päästeteenistuse seadus

0

Eesti Panga seadus

0

Teenetemärkide seadus

0

Erakonnaseadus

0

Vedelkütusevaru seadus

0

Turismiseadus

0

Meditsiiniseadme seadus

0

Geograafilise tähise

In [44]:
genai.configure(api_key=api_keys[0])
model = genai.GenerativeModel("gemini-1.5-pro")

prompt = (
    ["Sulle antakse mitme erineva mudeli vastused erinevatest seadustest, tsiteeri mulle mudeli vastuseid, mis pole '0'"] +
    [response_format] +
    responses + 
    ["Küsimus on järgnev:"] +
    [question]
)

best_answer = model.generate_content(prompt)

In [45]:
print(best_answer.text)

**Karistusseadustik**

Ei. Karistusseadustiku § 424 sätestab, et mootorsõiduki juhtimise eest joobeseisundis karistatakse rahalise karistuse või kuni kolmeaastase vangistusega.

**Autoveoseadus**

Ei, see on keelatud. Autoveoseaduse § 97 lõige 1 sätestab, et autojuht ei tohi olla alkoholi tarvitanud.



## TODO
hektel vastus tuleb umbes 15 minuti, aga me saaksime päringud teha paralleelselt, siis peaks vastus tulema umbes 2 minutiga.